In [47]:
import advent
from scipy.ndimage.filters import generic_filter
import numpy as np

data = advent.get_char_grid(11)
# Mapping: 0 = '.', 1 = 'L', -1 = '#'
smap = {'.': 0, 'L': 1, '#': 2}
data = np.array([[smap[c] for c in line] for line in data])

In [48]:
def apply(patch):
    # Given a flattened 3x3 grid, return what the center seat should become
    midx = len(patch) // 2
    if patch[midx] == smap['.']:
        return smap['.']
    elif patch[midx] == smap['L']:
        return smap['L'] if np.any(patch == smap['#']) else smap['#']
    elif patch[midx] == smap['#']:
        return smap['L'] if np.count_nonzero(patch == smap['#']) > 4 else smap['#']
    raise ValueError("Center of patch is {}".format(patch[midx]))

new_board = data.copy()
board = 0
turn_count = 0 
while not np.all(board == new_board):
    board = new_board
    new_board = generic_filter(new_board, apply, size=3, mode='constant', cval=0)
    turn_count += 1

# Now new_board == board
np.sum(board == smap['#']), turn_count

(2310, 101)

In [49]:
# For part 2,my approach is considering is that the nearest chair is the same every iteration
# So I find and save indexes of the nearest chair,...????????
from tqdm.notebook import trange


def apply(patch, size):
    # Given an odd-sized, square, flattened patch (at least 3x3)
    # Return what the center should become in part 2
    midx = size // 2
    patch = patch.reshape((size, size))
    if patch[midx, midx] == smap['.']:
        return smap['.']
    
    rot = np.rot90(patch)

    # The next 16 lines are quite ugly... sorry...
    topleft = np.diagonal(patch)[:midx][::-1]
    topleft = topleft[(topleft > smap['.']).argmax()]
    botright = np.diagonal(patch)[midx+1:]
    botright = botright[(botright > smap['.']).argmax()]

    botleft = np.diagonal(rot)[:midx][::-1]
    botleft = botleft[(botleft > smap['.']).argmax()]
    topright = np.diagonal(rot)[midx+1:]
    topright = topright[(topright > smap['.']).argmax()]

    top = patch[:midx,midx][::-1]
    top = top[(top > smap['.']).argmax()]
    bot = patch[midx+1:,midx]
    bot = bot[(bot > smap['.']).argmax()]

    left = patch[midx,:midx][::-1]
    left = left[(left > smap['.']).argmax()]
    right = patch[midx,midx+1:]
    right = right[(right > smap['.']).argmax()]
    # End of the ugly stuff!

    neighbors = sum(x == smap['#'] for x in [topleft, botright, botleft, topright, top, bot, left, right])

    #print(patch, neighbors, [topleft, botright, botleft, topright, top, bot, left, right], (patch[midx, midx] == smap['L'] and neighbors == 0) or (patch[midx, midx] == '#' and neighbors < 5))
    
    return smap['#'] if (patch[midx, midx] == smap['L'] and neighbors == 0) or (patch[midx, midx] == smap['#'] and neighbors < 5) else smap['L']



In [50]:
new_board = data.copy()
board = 0
# Size of filter. Must be big enough to see nearest chair in all directions.
# 21 gave the wrong answer, input size (91) will certainly be correct
size = 41  
assert size % 2 == 1, "Filter size must be odd"

for _ in trange(1000):
    if np.all(board == new_board):
        break

    board = new_board
    new_board = generic_filter(new_board, lambda x: apply(x, size), size=size, mode='constant', cval=0)

# Now new_board == board
if not np.all(board == new_board):
    print("Not converged in 1000 iterations!")
else:
    print(np.sum(board == smap['#']))

  0%|          | 0/1000 [00:00<?, ?it/s]

2074
